In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo
import json

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
username = 'jnordin'
date = '2018-09-24'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
minfilters = 2      # Reported detections in at least this many filters
minrefframes = 1    # Cut away alert information where this is not positive (how can it not be, but ok...)

marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = '/home/jnordin/local/github/AmpelSlackCosmology/'
slackfilepath = '/home/jnordin/Downloads/'

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sncandid_soup.json', 'r') as fp:
    soupids = json.load(fp)

# log. This should be saved and pushed to github. If everyone does this we should have the full scanning
# history for this run
logger = logging.getLogger(username)
handler = logging.FileHandler(logpath+'ztfcosmo_%s_%s.log'%(date,username))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)



In [3]:
df_sum = pd.read_csv(slackfilepath+"Summary_%s.csv"%(date))
df_phot = pd.read_csv(slackfilepath+"Photometry_%s.csv"%(date))

if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]
    logger.info("Cutting down alert info to subset with nrefframes keyword")

INFO:jnordin:Cutting down alert info to subset with nrefframes keyword


In [4]:
logger.info("There were %s candidates reported %s"%(df_sum.shape[0],date))

INFO:jnordin:There were 248 candidates reported 2018-09-24


In [5]:
# Find already classified SNe
i_untyped = (df_sum["T2-classification"]== "None") | (df_sum["T2-classification"].isnull())
df_classified = df_sum[ ~i_untyped]
df_tmp = df_sum[ i_untyped]
logger.info("Out of these %s already have Marshal type and will be skipped."%len(df_classified))


INFO:jnordin:Out of these 75 already have Marshal type and will be skipped.


In [6]:
# Are there any remaining candidates with spectroscopic redshifts above limit
i_highz = ( (df_tmp['T2-NEDz_z']>maxz) | (df_tmp['T2-milliquas_redshift']>maxz)  | (df_tmp['T2-SDSS_spec_z']>maxz) ) 
df_classified = df_tmp[ i_highz]
df_tmp2 = df_tmp[~i_highz]
logger.info("%s have a catalog z larger than %s  and will be skipped."%(np.sum(i_highz),maxz))


INFO:jnordin:11 have a catalog z larger than 0.1  and will be skipped.


In [7]:
# Which are already saved in the marshal?
i_saved = df_tmp2["T2-programs"].str.contains(marshal_saveprogs, regex=True, na=False, flags=re.IGNORECASE)
df_saved = df_tmp2[i_saved]
df_candidates = df_tmp2[~i_saved]
logger.info("Detected %s already saved candidates and %s new"%(len(df_saved),len(df_candidates)))

INFO:jnordin:Detected 16 already saved candidates and 146 new


In [8]:
# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_candidates.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        logger.info(msg)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        logger.info(msg)
        print(msg)
        continue

    age = np.max(jd)-np.min(jd)
    if age>maxage:
        msg = "%s: Skipping %s, age %s longer than limit %s"%(date,sne[1]["ztf_name"],age,maxage)
        logger.info(msg)
        print(msg)
        continue
        
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    if len(np.unique(phot["fid"]))<minfilters:
        msg = "%s: Skipping %s, only %s filters"%(date,sne[1]["ztf_name"],len(np.unique(phot["fid"])))
        logger.info(msg)
        print(msg)
        continue
        
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates to scan"%(len(sncandidates)))

INFO:jnordin:2018-09-24: Skipping ZTF18abwklar, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abtokcw, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abvgjgb, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abxbugc, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abvvokw, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18aajcqag, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18aamhnti, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abehrdw, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abtcdfv, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abvxbfh, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18aawrbuc, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abmsmnc, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18aaohtgx, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abuksoh, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipp

ZTF18abgoodz
ZTF18abslxpd
ZTF18abtmtfy
ZTF18aaupjtd
ZTF18abwklar
2018-09-24: Skipping ZTF18abwklar, less than 5 detections
ZTF18abbmenz
ZTF18abtokcw
2018-09-24: Skipping ZTF18abtokcw, less than 5 detections
ZTF18abwdnml
ZTF17aaaadqv
ZTF18abvgjgb
2018-09-24: Skipping ZTF18abvgjgb, less than 5 detections
ZTF18abxbugc
2018-09-24: Skipping ZTF18abxbugc, less than 5 detections
ZTF18abvvokw
2018-09-24: Skipping ZTF18abvvokw, less than 5 detections
ZTF18aajcqag
2018-09-24: Skipping ZTF18aajcqag, only 1 filters
ZTF18aamhnti
2018-09-24: Skipping ZTF18aamhnti, only 1 filters
ZTF18abnvotk
ZTF18abehrdw
2018-09-24: Skipping ZTF18abehrdw, less than 5 detections
ZTF18abmnvkx
ZTF18abvrzqj
ZTF18abufwxs
ZTF18abujamb
ZTF18abtcdfv
2018-09-24: Skipping ZTF18abtcdfv, never reached limit 19.5
ZTF18aawlhkh
ZTF18abvxbfh
2018-09-24: Skipping ZTF18abvxbfh, only 1 filters
ZTF18aawrbuc
2018-09-24: Skipping ZTF18aawrbuc, never reached limit 19.5
ZTF18abnvirx
ZTF18abmsmnc
2018-09-24: Skipping ZTF18abmsmnc, only 1 fi

INFO:jnordin:2018-09-24: Skipping ZTF18abujiho, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18aatycgd, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18aaimsgg, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abmsngt, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abfgygp, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abvbpku, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abmayrp, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abuyqbp, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abvglon, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18aburkuo, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abadbbb, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abkhqww, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abmwssp, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abwzedh, less than 5 detections
INFO:jnordin:2018-09-24: Ski

2018-09-24: Skipping ZTF18abwlpzf, less than 5 detections
ZTF18abujiho
2018-09-24: Skipping ZTF18abujiho, less than 5 detections
ZTF18aatycgd
2018-09-24: Skipping ZTF18aatycgd, less than 5 detections
ZTF18abmdnmi
ZTF18abqdwze
ZTF17aabihdn
ZTF18abulqzb
ZTF18aaimsgg
2018-09-24: Skipping ZTF18aaimsgg, only 1 filters
ZTF18absnsqx
ZTF18abmsngt
2018-09-24: Skipping ZTF18abmsngt, only 1 filters
ZTF18abfgygp
2018-09-24: Skipping ZTF18abfgygp, never reached limit 19.5
ZTF18abwcgyo
ZTF18abvbpku
2018-09-24: Skipping ZTF18abvbpku, only 1 filters
ZTF18abmayrp
2018-09-24: Skipping ZTF18abmayrp, only 1 filters
ZTF18abuyqbp
2018-09-24: Skipping ZTF18abuyqbp, never reached limit 19.5
ZTF18ablllyw
ZTF18aajswer
ZTF18aampiwv
ZTF18ablprcf
ZTF18abtswjk
ZTF18aaunlts
ZTF18abvglon
2018-09-24: Skipping ZTF18abvglon, less than 5 detections
ZTF18abwqbxc
ZTF18aburkuo
2018-09-24: Skipping ZTF18aburkuo, never reached limit 19.5
ZTF18abvytad
ZTF18abulemm
ZTF18abadbbb
2018-09-24: Skipping ZTF18abadbbb, never reached l

INFO:jnordin:2018-09-24: Skipping ZTF18abxbmqh, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abshgbq, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abvvmvw, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abuwqdi, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abcrcmn, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18aaxujmt, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18aabvpbb, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abvjzdz, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abecjrr, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abwlsoi, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abvoydx, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abdklhh, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abtkzfi, only 1 filters
INFO:jnordin:2018-09-24: Skipping ZTF18abvgivy, less than 5 detections
INFO:jnordin:2018-09

2018-09-24: Skipping ZTF18abwugyc, less than 5 detections
ZTF18abtghbi
ZTF18abxbmqh
2018-09-24: Skipping ZTF18abxbmqh, less than 5 detections
ZTF18aampjcm
ZTF18abvlkyj
ZTF18abshgbq
2018-09-24: Skipping ZTF18abshgbq, never reached limit 19.5
ZTF18abvvmvw
2018-09-24: Skipping ZTF18abvvmvw, less than 5 detections
ZTF18abporae
ZTF18abvxdou
ZTF18abuwqdi
2018-09-24: Skipping ZTF18abuwqdi, never reached limit 19.5
ZTF18aaxudip
ZTF18abcrcmn
2018-09-24: Skipping ZTF18abcrcmn, only 1 filters
ZTF18abnuzzt
ZTF18aaxujmt
2018-09-24: Skipping ZTF18aaxujmt, never reached limit 19.5
ZTF18aabvpbb
2018-09-24: Skipping ZTF18aabvpbb, less than 5 detections
ZTF18abvjzdz
2018-09-24: Skipping ZTF18abvjzdz, less than 5 detections
ZTF18absgvqb
ZTF18abecjrr
2018-09-24: Skipping ZTF18abecjrr, only 1 filters
ZTF18abwlsoi
2018-09-24: Skipping ZTF18abwlsoi, less than 5 detections
ZTF18aaqovfv
ZTF18abvoydx
2018-09-24: Skipping ZTF18abvoydx, never reached limit 19.5
ZTF18abdklhh
2018-09-24: Skipping ZTF18abdklhh, neve

INFO:jnordin:2018-09-24: Skipping ZTF18abxxviy, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abymhpa, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abyymiw, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abryobq, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abtnhlw, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abxzwkh, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abxxssh, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abyckee, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abygcwk, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abxyvdr, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abscmgg, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abxzcky, less than 5 detections
INFO:jnordin:2018-09-24: Skipping ZTF18abyfowk, never reached limit 19.5
INFO:jnordin:2018-09-24: Skipping ZTF18abuiiqm, less than 5 detec

2018-09-24: Skipping ZTF18abxelyl, less than 5 detections
ZTF18abxxviy
2018-09-24: Skipping ZTF18abxxviy, less than 5 detections
ZTF18abymhpa
2018-09-24: Skipping ZTF18abymhpa, never reached limit 19.5
ZTF18abyymiw
2018-09-24: Skipping ZTF18abyymiw, never reached limit 19.5
ZTF18abryobq
2018-09-24: Skipping ZTF18abryobq, less than 5 detections
ZTF18abtnhlw
2018-09-24: Skipping ZTF18abtnhlw, never reached limit 19.5
ZTF18abxzwkh
2018-09-24: Skipping ZTF18abxzwkh, never reached limit 19.5
ZTF18abxxssh
2018-09-24: Skipping ZTF18abxxssh, less than 5 detections
ZTF18abyckee
2018-09-24: Skipping ZTF18abyckee, less than 5 detections
ZTF18abygcwk
2018-09-24: Skipping ZTF18abygcwk, less than 5 detections
ZTF18abxyvdr
2018-09-24: Skipping ZTF18abxyvdr, less than 5 detections
ZTF18abscmgg
2018-09-24: Skipping ZTF18abscmgg, never reached limit 19.5
ZTF18abxzcky
2018-09-24: Skipping ZTF18abxzcky, less than 5 detections
ZTF18abyfowk
2018-09-24: Skipping ZTF18abyfowk, never reached limit 19.5
ZTF18ab

In [9]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Quick plot of SN nbr k
    i : {0:undecided, 1:submit, 2:wait 3:bogus}
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = sncandidates[snnbr]
    if i == 'Wait':
        logger.info("Waiting for further obs for SN %s %s"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Submit':
        logger.info("Sending SN %s %s for possible follow-up"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'Garbage':
        logger.info("I never want to see SN %s %s again!"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(sncandidates):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    # Reset    
    snname = sncandidates[snnbr]
    

    phot = df_phot[ df_phot["ztf_name"]==snname ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']


    plt.title(snname + ' ' + str(snnbr) + ' out of '+str(len(sncandidates)))
    plt.plot(jd[filters==1]-Time.now().jd,magpsf[filters==1],'go')
    plt.plot(jd[filters==2]-Time.now().jd,magpsf[filters==2],'ro')
    plt.plot(jd[filters==3]-Time.now().jd,magpsf[filters==3],'ko')
    plt.gca().invert_yaxis()
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)
wiggy = widgets.RadioButtons(
    options=['Nothing','Wait', 'Submit', 'Garbage','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [10]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [11]:
# This is the all important list of decisions you have made
decisions = {}

In [12]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

interactive(children=(RadioButtons(description='Action:', options=('Nothing', 'Wait', 'Submit', 'Garbage', 'Go…

In [13]:
# Now fun starts for real. Lets download all the candidates and check whether we find all the candidate ids there
# (cause are the only that are useful for saving). This will take a while.
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

DEBUG:marshaltools.ProgramList:listing accessible programs
DEBUG:marshaltools.ProgramList:Starting list_programs.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_programs.cgi?None
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
DEBUG:marshaltools.ProgramList:Starting list_program_sources.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_program_sources.cgi?programidx=4&getredshift=1&getclassification=1
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Loaded 184 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-26 08:00:20 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration 

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:Query from 2018-03-31 00:00:00.000 to 2018-04-05 00:00:00.000 returned 0 candidates. Total: 0
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-05-30 00:00:00 and 2018-06-04 00:00:00
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-03-01+00%3A00%3A00&enddate=2018-03-06+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-04-25+00%3A00%3A00&enddate=2018-04-30+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:Successf

DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-04 00:00:00 and 2018-07-09 00:00:00
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-14 00:00:00 and 2018-07-19 00:00:00
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-19 00:00:00 and 2018-07-24 00:00:00
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-24 00:00:00 and 2018-07-29 00:00:00
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-29 00:00:00 and 2018-08-03 00:00:00
DEBUG:marshaltools.ProgramList:Start

DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-08-23+00%3A00%3A00&enddate=2018-08-28+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 186 sources
DEBUG:marshaltools.ProgramList:Query from 2018-08-23 00:00:00.000 to 2018-08-28 00:00:00.000 returned 186 candidates. Total: 414
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-07-24+00%3A00%3A00&enddate=2018-07-29+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 74 sources
DEBUG:marshaltools.ProgramList:Query from 2018-07-24 00:00:00.000 to 2018-07-29 00:00:00.000 returned 74 candidates. Total: 488
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_ca

In [14]:
ingestoldcand_start = Time.now()
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    logger.info("Trying to ingest %s through avroid %s"%(sncandidates[snnbr],avroid))
    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        logger.info('... already a candidate with id %s'%(canddate['candid']))
        continue
    
    pl.ingest_avro(avroid)
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

INFO:jnordin:Trying to ingest ZTF18aaupjtd through avroid 630258045315015003
DEBUG:marshaltools.ProgramList:can't find source named ZTF18aaupjtd among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find it among candidates either.
INFO:marshaltools.ProgramList:Ingesting avro package 630258045315015003 into program AMPEL Test
DEBUG:marshaltools.ProgramList:Starting ingest_avro_id.cgi post. Attempt # 0


Starting ingesting at 2018-09-26 08:01:55.706269 (useful if you want to check scanning page directly)


DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/ingest_avro_id.cgi?programidx=4&avroid=630258045315015003
DEBUG:marshaltools.ProgramList:Successful growth connection.
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Trying to ingest ZTF18abmnvkx through avroid 631231962115015004
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abmnvkx among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find it among candidates either.
INFO:marshaltools.ProgramList:Ingesting avro package 631231962115015004 into program AMPEL Test
DEBUG:marshaltools.ProgramList:Starting ingest_avro_id.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/ingest_avro_id.cgi?programidx=4&avroid=631231962115015004
DEBUG:marshaltools.ProgramList:Successful growth connection.
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Trying to ingest 

Stoppedingesting at 2018-09-26 08:02:24.644641 (useful if you want to check scanning page directly)


In [15]:
# We do this again because it is soo funny
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

DEBUG:marshaltools.ProgramList:listing accessible programs
DEBUG:marshaltools.ProgramList:Starting list_programs.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_programs.cgi?None
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
DEBUG:marshaltools.ProgramList:Starting list_program_sources.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_program_sources.cgi?programidx=4&getredshift=1&getclassification=1
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Loaded 184 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-26 08:05:19 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration 

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:Query from 2018-04-05 00:00:00.000 to 2018-04-10 00:00:00.000 returned 0 candidates. Total: 0
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-05-30 00:00:00 and 2018-06-04 00:00:00
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-04-10+00%3A00%3A00&enddate=2018-04-15+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:Query from 2018-04-10 00:00:00.000 to 2018-04-15 00:00:00.000 returned 0 candidates. Total: 0
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL T

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-07-29 00:00:00 and 2018-08-03 00:00:00
DEBUG:marshaltools.ProgramList:Query from 2018-05-20 00:00:00.000 to 2018-05-25 00:00:00.000 returned 0 candidates. Total: 1
DEBUG:marshaltools.ProgramList:Starting list_candidates.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-06-19+00%3A00%3A00&enddate=2018-06-24+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 0 sources
DEBUG:marshaltools.ProgramList:querying scanning page of program AMPEL Test (scienceprogram 42) for ingested times between 2018-08-03 00:00:00 and 2018-08-08 00:00:00
DEBUG:marshaltools.ProgramList

DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 162 sources
DEBUG:marshaltools.ProgramList:Query from 2018-08-03 00:00:00.000 to 2018-08-08 00:00:00.000 returned 162 candidates. Total: 390
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-08-23+00%3A00%3A00&enddate=2018-08-28+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retrieved 186 sources
DEBUG:marshaltools.ProgramList:Query from 2018-08-23 00:00:00.000 to 2018-08-28 00:00:00.000 returned 186 candidates. Total: 576
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_candidates.cgi?scienceprogram=42&startdate=2018-08-08+00%3A00%3A00&enddate=2018-08-13+00%3A00%3A00&showSaved=all
DEBUG:marshaltools.ProgramList:Successful growth connection.
DEBUG:marshaltools.ProgramList:retr

In [16]:
# And we do this again as well. If this worked we should now have all candidates saved
togrowth = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        continue
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    msg = "Failed to ingest alert %s for SN %s"%(avroid,sncandidates[snnbr])
    logger.info(msg)
    togrowth.append(msg)
    


DEBUG:marshaltools.ProgramList:can't find source named ZTF18aaupjtd among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18aaupjtd among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abmnvkx among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abmnvkx among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18absoomk among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18absoomk among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18abvnwzz among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abvnwzz among candidates of program AMPEL Test
DEBUG:marshaltools.ProgramList:found source ZTF18abwntpz among saved sources of program AMPEL Test
DEBUG:marshaltools.ProgramList:can't find source named ZTF18ablllyw among sav

In [17]:
# Write an email to Mansi/Growth team with the following content
print(togrowth)
# And when we hear back from them this SN should be saved. I hope they do it for us

[]


In [29]:
# Ok - this does not work since we have the wrong candidate id...
missing_candidsne = []
# Now it is time to save remaining candidates
savecand_start = Time.now()
print("Starting saving at %s (not really that useful)"%(savecand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    try:
        candid = soupids[sncandidates[snnbr]]
    except KeyError:
        logger.info("Lacking candid for %s"%(sncandidates[snnbr]))
        missing_candidsne.append(snnbr)
    logger.info("Try to save %s through candid %s"%(sncandidates[snnbr],candid))
    pl.save_source(candid,42)


        
savecand_end = Time.now()
print("Stopped saving at %s"%(savecand_end))

INFO:jnordin:Try to save ZTF18aaupjtd through candid 1548525
INFO:marshaltools.ProgramList:Saving source 1548525 into program 42
DEBUG:marshaltools.ProgramList:Starting save_cand_growth.cgi post. Attempt # 0


Starting saving at 2018-09-26 08:23:43.501558 (not really that useful)


DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/save_cand_growth.cgi?program=42&candid=1548525
DEBUG:marshaltools.ProgramList:Successful growth connection.
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18abmnvkx through candid 1543304
INFO:marshaltools.ProgramList:Saving source 1543304 into program 42
DEBUG:marshaltools.ProgramList:Starting save_cand_growth.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/save_cand_growth.cgi?program=42&candid=1543304
DEBUG:marshaltools.ProgramList:Successful growth connection.
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18absoomk through candid 1548864
INFO:marshaltools.ProgramList:Saving source 1548864 into program 42
DEBUG:marshaltools.ProgramList:Starting save_cand_growth.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech

Stopped saving at 2018-09-26 08:24:01.968128


In [30]:
# Check whether we have missing candidate ids
print("Missing candidate id for these sne: %s"%(missing_candidsne))

Missing candidate id for these sne: []


In [31]:
# Reload saved sources
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

DEBUG:marshaltools.ProgramList:listing accessible programs
DEBUG:marshaltools.ProgramList:Starting list_programs.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_programs.cgi?None
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
DEBUG:marshaltools.ProgramList:Starting list_program_sources.cgi post. Attempt # 0
DEBUG:marshaltools.ProgramList:request URL: http://skipper.caltech.edu:8080/cgi-bin/growth/list_program_sources.cgi?programidx=4&getredshift=1&getclassification=1
DEBUG:marshaltools.ProgramList:Successful growth connection.
INFO:marshaltools.ProgramList:Loaded 192 saved sources for program AMPEL Test.


In [32]:
# Loop through them again to check they are all saved
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(sncandidates[snnbr]))
        continue

    logger.info('SN %s was saved'%(sncandidates[snnbr]))


DEBUG:marshaltools.ProgramList:found source ZTF18aaupjtd among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18aaupjtd was saved
DEBUG:marshaltools.ProgramList:found source ZTF18abmnvkx among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18abmnvkx was saved
DEBUG:marshaltools.ProgramList:found source ZTF18absoomk among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18absoomk was saved
DEBUG:marshaltools.ProgramList:found source ZTF18abvnwzz among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18abvnwzz was saved
DEBUG:marshaltools.ProgramList:found source ZTF18abwntpz among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18abwntpz was saved
DEBUG:marshaltools.ProgramList:found source ZTF18ablllyw among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18ablllyw was saved
DEBUG:marshaltools.ProgramList:found source ZTF18abtswjk among saved sources of program AMPEL Test
INFO:jnordin:SN ZTF18abtswjk was saved
DEBUG:marshaltools.ProgramList:fou

In [24]:
## Hopefully done. If all of the candidates were saved (no ojojs) you are done with scanning!
# If candidate IDs are still missing you have to try to scrape the webpages for them as below. Have fun!

In [25]:
# Trying to get candid through the seargant
from ampel.pipeline.t3.sergeant import marshal_functions
tbase = Time.now()
allcandidates = []

In [26]:
for k in range(2):
    print(k)
    tend = Time(tbase.jd-k,format='jd')
    tstart = Time(tbase.jd-k-1,format='jd')
    ser = marshal_functions.Sergeant(program_name='AMPEL Test',start_date=tstart.iso,end_date=tend.iso)
    ser.cutprogramidx = 42
    cands = ser.list_scan_sources()
    print("Found %s cands"%(len(cands)))
    allcandidates.extend(cands)

0


DEBUG:ampel.pipeline.t3.sergeant.marshal_functions:start_date : 2018-09-25 08:20:23.131
DEBUG:ampel.pipeline.t3.sergeant.marshal_functions:end_date   : 2018-09-26 08:20:23.131


Found 84 cands
1


DEBUG:ampel.pipeline.t3.sergeant.marshal_functions:start_date : 2018-09-24 08:20:23.131
DEBUG:ampel.pipeline.t3.sergeant.marshal_functions:end_date   : 2018-09-25 08:20:23.131


Found 6 cands


In [28]:
for cand in allcandidates:
    if cand["name"] in soupids.keys():
        continue
    print("new candidate %s"%(cand["name"]))
    soupids[cand["name"]] = cand["candid"]

In [34]:
# Save the added dict of candidate id
with open('sncandid_soup.json', 'w') as fp:
    json.dump(soupids, fp)

In [ ]:
# cheat for 09 23
decisions = {0: 'Garbage', 1: 'Garbage', 2: 'Garbage', 3: 'Garbage', 4: 'Wait', 5: 'Garbage', 6: 'Submit', 7: 'Submit', 8: 'Garbage', 9: 'Garbage', 10: 'Garbage', 11: 'Garbage', 12: 'Garbage', 13: 'Garbage', 14: 'Garbage', 15: 'Garbage', 16: 'Submit', 17: 'Submit', 18: 'Submit', 19: 'Garbage', 20: 'Garbage', 21: 'Garbage', 22: 'Garbage', 23: 'Garbage', 24: 'Garbage', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Garbage', 30: 'Submit', 31: 'Submit', 32: 'Garbage', 33: 'Submit', 34: 'Submit', 35: 'Submit', 36: 'Garbage', 37: 'Submit', 38: 'Garbage', 39: 'Garbage', 40: 'Garbage'}